# Regression

Unlike with **classification** where we are predicting a **nominal** label, in **regression** problems we are predicting a **numerical** quantity.

The simplest form of regression is **linear regression** (also known as the **least squares method**). This is a problem from calculus that tries to find a line that minimizes the squared distance from a collection of points.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Linear_regression.svg/200px-Linear_regression.svg.png)

![](https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/Linear_least_squares_example2.png/190px-Linear_least_squares_example2.png)

From Wikipedia: `In linear regression, the observations (red) are assumed to be the result of random deviations (green) from an underlying relationship (blue) between a dependent variable (y) and an independent variable (x).`

The method also handles fitting lines in multidimensional data.

In [ ]:
# Load our prerequisites
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [ ]:
# Download solution(s)

import urllib.request
import os

def download_data(path):
    if os.path.exists(path):
        return
    if not os.path.exists('solutions'):
        os.mkdir('solutions')
    url = 'https://raw.githubusercontent.com/ualberta-rcg/python-machine-learning/main/notebooks/' + path
    output_file = path
    urllib.request.urlretrieve(url, output_file)
    print("Downloaded " + path)

def show_solution(file):
    fp = open('solutions/{}'.format(file), 'r')
    print(fp.read())

download_data('solutions/real-estate-san-fran.py')

## A problem: California Housing Prices

Scikit-learn comes with a number of datasets (or will fetch the data) that are good for exploring techiniques in data science.

The dataset we will fetch relates to California Housing prices from the 1990 census, where the data is organized in blocks groups (geographically close, between 600 - 3000 houses in each block).

Our task will be to predict house prices (the label or target) given other features in the data.

We fetch the data ...

In [ ]:
import sklearn.datasets

california = sklearn.datasets.fetch_california_housing()
california

---

The data downloaded is a dictionary, which includes the housing data and some metadata. We can take a look at the description that comes with this download:

In [ ]:
print(california['DESCR'])

In [ ]:
california.keys()

We can assemble a dataframe from the various components downloaded. The downloaded data is already separated into feature and labels, so we will combine both into our dataframe.

In [ ]:
california_df = pd.DataFrame(california['data'], columns=california['feature_names'])
california_df['MedHouseVal'] = california['target']

In [ ]:
california_df.head()

## Exploratory Data Analysis (EDA)

This is a neat scatter plot of the data, mapping the longiture/latitude, making the size relative to the population of the blocks, and coloring based on the median house values.

In [ ]:
california_df.plot(kind="scatter", x="Longitude", y="Latitude",
                   alpha=0.4,
                   s=california_df["Population"]/100,
                   label="Population",
                   c="MedHouseVal", 
                   figsize=(12,8),
                   cmap=plt.get_cmap("jet"),
                   colorbar=True)

---

We can visualize how some of the variables vary with each other using the Pandas function [`scatter_matrix`](https://pandas.pydata.org/docs/reference/api/pandas.plotting.scatter_matrix.html).

Here we will look at how median housing value varies with:
* median income
* average number of rooms
* house age

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ["MedHouseVal", "MedInc", "AveRooms", "HouseAge"]
scatter_matrix(california_df[attributes], figsize=(12, 8))

## Outlier removal

Looking at the above, we some very sparce data values at the high ends of some of these graphs.

Lets look a little further.

In [ ]:
california_df.describe()

---

There are no correct answers when deciding how to deal with outliers.

We'll decide to throw out any data that is **greater than three standard deviations** away from the mean for a variable.

Lets look at the standard deviation of median income:

In [ ]:
california_df.MedInc.std()

The upper cut off value for median income (above which the data is declared to be an outlier) is:

In [ ]:
income_cut_off = california_df.MedInc.mean() + 3 * california_df.MedInc.std()
income_cut_off

The rows in the dataset that fall within the acceptable range are:

In [ ]:
california_df.MedInc < income_cut_off

We can now replace our dataset with one that only includes these rows:

In [ ]:
california_df = california_df[california_df.MedInc < income_cut_off]

Looking at the dataset again ...

In [ ]:
california_df.describe()

## Exercise: remove the outliers from the average rooms data

Use three standard deviations from the mean as your guide. (See the **Putting it all together** section below for a possible solution)

In [ ]:
# Your code here ...

---

We can look at the descriptive statistics and the `scatter_matrix` again ...

In [ ]:
california_df.describe()

In [ ]:
scatter_matrix(california_df[attributes], figsize=(12, 8))

## Selecting features and labels

In [ ]:
X = california_df[["MedInc", "AveRooms", "HouseAge"]]
y = california_df["MedHouseVal"]

## Spliting the data into training and test data sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
X_train

## Selecting a model and training on the training data

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model = model.fit(X_train, y_train)

## Making predictions

Again, we use the `predict` method to make predictions on some features.

In [ ]:
predictions = model.predict(X_test)

In [ ]:
# Checking out the first few predictions

print(predictions[:10])
print(list(y_test.head(10)))

## Evaluating regression models

There are some measures we can use for evaluating the quality of our predictions

### Mean Absolute Error (MAE)

$MAE = (\sum_N |actual_i - predicted_i|) / N$

### Mean Squared Error (MSE)

$MSE = \sum_N (actual_i - predicted_i)^2 / N$

### Coefficient of Determination ($R^2$)

$R^2 = $ (math goes here)

* 1 is a perfect score, when all predictions are correct (interpretation: the model explains all the variability of the labels around their mean)
* If your model only every predicted the mean of the labels, you would get a score of zero (interpretation: the model explains none of the variability of the labels around their mean)
* Can be less than zero

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print('Mean Absolute Error:', mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', mean_squared_error(y_test, predictions))
print('R2 Score:', r2_score(y_test, predictions))

---

What do you think of this model? Did we explain all of the variability in housing prices?

Unfortunately, pricing houses can be complicated ...

![](https://i.insider.com/4fb69a54eab8ea195000000d?width=750&format=jpeg&auto=webp)

---

Question: Are there any other features (either included with the data, or engineered) you might use to help improve the model?

## Putting it all together

The code is pretty scattered in this notebook. Lets look at the pipeline in it's entirety:

In [ ]:
import sklearn
import sklearn.datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Set a random seed for reproducability
np.random.seed(1337)

# Get/load the data into a dataframe ...
california = sklearn.datasets.fetch_california_housing()
california_df = pd.DataFrame(california['data'], columns=california['feature_names'])
california_df['MedHouseVal'] = california['target']

# Remove outliers ...
income_cut_off = california_df.MedInc.mean() + 3 * california_df.MedInc.std()
california_df = california_df[california_df.MedInc < income_cut_off]

room_cut_off = california_df.AveRooms.mean() + 3 * california_df.AveRooms.std()
california_df = california_df[california_df.AveRooms < room_cut_off]

# Choose features/labels
X = california_df[["MedInc", "AveRooms", "HouseAge"]]
y = california_df["MedHouseVal"]

# Split data into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# Select model and train it
model = LinearRegression()
model = model.fit(X_train, y_train)

# Make prediction
predictions = model.predict(X_test)

# Evaluate predictions
print('Mean Absolute Error:', mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', mean_squared_error(y_test, predictions))
print('R2 Score:', r2_score(y_test, predictions))

## Retrieving the parameters of the linear model

The linear coeffients (i.e., the gradient or slope) are in the `model.coef_` attribute of the model:

In [ ]:
model.coef_

The interecept is in `model.intercept_`:

In [ ]:
model.intercept_

We can calculate the models prediction by hand for the first row of data by taking the dot product of the coefficients and adding the intercept ...

In [ ]:
x = X_test.iloc[0].values
x

In [ ]:
x.dot(model.coef_) + model.intercept_

Comparing this with the output from `model.predict` ...

In [ ]:
predictions[0]

## Other linear regression models

Scikit-learn provides a number of other [variations](
https://scikit-learn.org/stable/modules/linear_model.html) on linear regression you can try for fitting data.

## Gradient Boosting

Decision Tree Classifiers can be combined into the ensemble model Random Forest. Similarly, we can also use ensemble models for regression tasks. One class that impliments this in Scikit-learn is [`sklearn.ensemble.GradientBoostingRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html).

The idea behind this algorithm is a bit complicated, with a lot of low-level details. I asked some of my Compute Canada colleagues what they thought of this following desciption, intended for a novice audience:

> We create an accurate model (a **"strong learner"**) by combining the inputs of several less accurate models (or **"weak learners"**). In Scikit-learn, these weak learners are instances of `DecisionTreeRegressor`. Using a step-by-step approach, the weak learners are added to the ensemble, and the weights that determine each weak learner's contribution are tweaked via an optimization algorithm.

My colleague Mat suggested the following additional information:

> The other way you could go about it is that we have 5 kids, johnny is good at x, suzie is good at y, and so on, and we defer to the kid who knows the most about a specific problem.
>
> So each kid gets to put in an input based on how well they know that sort of problem
>
> I would lean into anthropomorphizing the problem, and treating the model like a team of people who don't necessarily have the whole answer themselves, but working together, they perform much better. You can talk about the weighting algorithm as the "supervisor" or "manager" of the team

My colleague Lucas also added:

> I assume they're familiar with the concept of error when you get to talking about grad boosting? If so I'd say something like: Train a simple decision tree. That tree will make a certain number of prediction errors. Take those errors and train another tree on them. Add the "correction" to the original tree. This new "corrected" tree will also make some prediction errors. Take those and train another tree... keep doing that, "compensating" for the errors made by your tree with a subsequent tree, until convergence or you're satisfied with a (hopefully low) error rate

The nice thing is that you don't need to know too much about the low level details to implement a pipeline using this algorithm.

## Exercise

Implement a regression pipeline using the `GradientBoostingRegressor`.

Look at the linked documentation for the default values of `n_estimators`, `max_depth`, and `learning_rate`. Feel free to modify the parameters.

In [ ]:
### Your code here ...

---

## Exercise

In our real estate example, engineer a feature that represents the distance from San Fransisco, and set up a regression pipeline. Can you think of any other similar features that might help boost the performance of our model?

In [ ]:
### Your code here ...

To see one possible solution ...

In [ ]:
# PRINT SOLUTION (copy/paste output into a cell to run)
show_solution('real-estate-san-fran.py')

---

On to the next notebook, on [unsupervised learning](03-unsupervised.ipynb).